
---

# Task 1: Sentence Transformer Implementation for Netflix App Reviews

In this model, we aim to generate meaningful embeddings from Netflix app reviews. The process begins with essential data preprocessing steps and progresses through the use of a transformer model. Below are the detailed steps and rationales:

## Data Preprocessing
Before processing the reviews through our model, we first ensure the data is clean and well-structured:
- **Cleaning Null or NA Elements:** We remove any null or NA values from the dataset to prevent any issues during processing. This step is crucial as missing values can lead to errors in data handling and model training, and may adversely affect the model's performance.
- **Enforcing Data Types on Columns:** Specifying explicit data types for each column improves both the efficiency and accuracy of operations performed on the data. It ensures that operations such as sorting, querying, and arithmetic computations are performed consistently and correctly.

## Tokenization
For tokenization, we utilize a **Byte Pair Encoding (BPE) Tokenizer**:
- **Choice of BPE Tokenizer:** The BPE tokenizer is chosen for its efficiency in managing vocabulary size and its ability to handle out-of-vocabulary words gracefully. It works by iteratively merging the most frequent pairs of bytes (or characters) in a sequence, which is particularly effective for the diverse linguistic content found in app reviews. This method helps in reducing the sparsity of the dataset and ensures that the model can interpret and process a wide variety of textual inputs without a massive, unwieldy vocabulary.

## Transformer Model
Once the data is tokenized, it is fed into a transformer model to generate embeddings:
- **Transformer Blocks:** The core of our transformer model consists of several layers built out of transformer blocks. Each block contains a multi-head attention mechanism paired with a position-wise feedforward network.
  - **Positional Encoding:** Unlike recurrent neural networks, transformers do not inherently process data in sequence. Positional encodings are therefore used to inject some information about the order of the tokens in the sequence. This is vital for our task as the order of words in reviews can change the sentiment and meaning dramatically.
  - **Multi-Head Attention:** This mechanism allows the model to jointly attend to information from different representation subspaces at different positions. With this, the model can capture a richer context and understand nuanced relationships between words in reviews, which is crucial for generating high-quality embeddings.

## Rationale for Architecture Choice
The choice of a transformer model equipped with positional encodings and multi-head attention is driven by the need to understand the contextual relationships between words in textual data effectively. Transformers provide significant advantages in terms of training speed and effectiveness, especially on large datasets like Netflix reviews, due to their parallelizable architecture and the ability to capture long-range dependencies between words.

This architecture ensures that the embeddings generated are contextually enriched, making them suitable for further tasks such as sentiment analysis, recommendation systems, or trend analysis based on user reviews.

---


# Data Loading

In [22]:
import pandas as pd

df = pd.read_csv('./archive/netflix_reviews.csv')

# as we only want to display the generation of embeddings, we will only use the first 1000 rows

# Data Cleaning

In [23]:
df = df[['content','score','thumbsUpCount','at']]

df = df[~df['content'].isnull()]

df = df.astype({
    'score':'int16',
    'thumbsUpCount':'int16'
})

df['at'] = pd.to_datetime(df['at']) 
df['at'] = df['at'].dt.strftime('%Y')

# Tokenization

In [24]:
df['content'].to_csv('content_text.txt', index=False, header=False)

In [25]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

tokenizer = Tokenizer(BPE())
tokenizer.pre_tokenizer = Whitespace()
trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
tokenizer.train(files=["content_text.txt"], trainer=trainer)

In [26]:
from tqdm import tqdm

def tokenize(text):
    return tokenizer.encode(text).ids

tqdm.pandas(desc="Processing dataframe: ")
df['tokenized_content'] = df['content'].progress_apply(lambda x: tokenize(str(x)))
df['tokenized_content'].head()

Processing dataframe: 100%|██████████| 112730/112730 [00:08<00:00, 12979.63it/s]


0                                                                                                                                                                                                                                                                                                                                                                                                                              [56, 12780, 1677, 20724, 6867, 5390, 1522, 70, 2076]
1    [46, 7805, 2049, 17, 1530, 1607, 46, 12, 82, 2430, 3906, 1775, 19, 1525, 2521, 2526, 70, 7, 2283, 7, 2705, 1488, 1607, 2019, 2407, 19, 46, 1533, 12, 89, 1601, 1474, 1525, 1872, 46, 12, 82, 3863, 17, 1477, 70, 2216, 12, 88, 2099, 17, 1486, 1645, 1605, 46, 12, 82, 1599, 1815, 1518, 2145, 1486, 70, 4260, 2331, 1678, 1476, 1753, 1476, 2572, 1702, 1480, 3082, 1483, 1515, 1525, 2243, 1483, 2095, 1476, 2283, 2540, 1488, 1716, 1905, 1529, 1863, 1603, 2887, 1529, 19]
2                                                               

In [27]:
df = df[['content','tokenized_content','score','thumbsUpCount','at']]

# Multi Head Attention 

In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MultiHeadAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(MultiHeadAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert self.head_dim * heads == embed_size, "Embed size needs to be divisible by heads"

        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)

    def forward(self, values, keys, query, mask):
        N = query.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        queries = query.reshape(N, query_len, self.heads, self.head_dim)

        values = self.values(values)
        keys = self.keys(keys)
        queries = self.queries(queries)

        attention = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])

        if mask is not None:
            attention = attention.masked_fill(mask == 0, float("-1e20"))

        attention = torch.softmax(attention / (self.embed_size ** (1 / 2)), dim=3)

        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.heads * self.head_dim
        )

        out = self.fc_out(out)
        return out


# Positional Encoding

In [29]:
class PositionalEncoding(nn.Module):
    def __init__(self, embed_size, max_len=500):
        super(PositionalEncoding, self).__init__()
        self.encoding = torch.zeros(max_len, embed_size)
        pos = torch.arange(0, max_len).unsqueeze(1).float()
        _2i = torch.arange(0, embed_size, step=2).float()

        self.encoding[:, 0::2] = torch.sin(pos / (10000 ** (_2i/embed_size)))
        self.encoding[:, 1::2] = torch.cos(pos / (10000 ** (_2i/embed_size)))
        self.encoding = self.encoding.unsqueeze(0)

        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.encoding = self.encoding.to(self.device)  

    def forward(self, x):
        return self.encoding[:, :x.size(1), :].to(x.device)

# Transformer block

In [30]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()
        self.attention = MultiHeadAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size)
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        attention = self.attention(value, key, query, mask)

        x = self.dropout(self.norm1(attention + query))
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))
        return out

# Transformer

In [31]:
class TransformerForEmbeddings(nn.Module):
    def __init__(self, src_vocab_size, embed_size, num_layers, heads, forward_expansion, dropout, max_length):
        super(TransformerForEmbeddings, self).__init__()
        self.embed_size = embed_size
        self.word_embedding = nn.Embedding(src_vocab_size, embed_size)
        self.positional_encoding = PositionalEncoding(embed_size, max_length)

        self.layers = nn.ModuleList([
            TransformerBlock(embed_size, heads, dropout, forward_expansion) for _ in range(num_layers)
        ])

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = self.dropout(self.word_embedding(x) + self.positional_encoding(x))
        for layer in self.layers:
            out = layer(out, out, out, None)

        sentence_embedding = out.mean(dim=1)  # Mean pooling
        return sentence_embedding

In [32]:
max_length = 128
batch_size = 32
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
def pad_sequences(vector, pad_idx, max_len):
    padded = vector + [pad_idx] * (max_len - len(vector))
    return padded[:max_len]

pad_idx = tokenizer.token_to_id("[PAD]")
df['padded_tokens'] = df['tokenized_content'].progress_apply(lambda x: pad_sequences(x, pad_idx, max_length))
padded_tokens_tensor = torch.tensor(list(df['padded_tokens']))

Processing dataframe: 100%|██████████| 112730/112730 [00:00<00:00, 170408.11it/s]


In [15]:
from torch.utils.data import TensorDataset, DataLoader

dataset = TensorDataset(padded_tokens_tensor)
loader = DataLoader(dataset, batch_size=32, shuffle=False)

In [18]:
vocab_size = tokenizer.get_vocab_size()

model = TransformerForEmbeddings(src_vocab_size=vocab_size, 
                    embed_size=128,
                    num_layers=4,
                    heads=8,
                    forward_expansion=4,
                    dropout=0.1,
                    max_length=max_length
                )

model.to(device)

TransformerForEmbeddings(
  (word_embedding): Embedding(30000, 128)
  (positional_encoding): PositionalEncoding()
  (layers): ModuleList(
    (0-3): 4 x TransformerBlock(
      (attention): MultiHeadAttention(
        (values): Linear(in_features=16, out_features=16, bias=False)
        (keys): Linear(in_features=16, out_features=16, bias=False)
        (queries): Linear(in_features=16, out_features=16, bias=False)
        (fc_out): Linear(in_features=128, out_features=128, bias=True)
      )
      (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (feed_forward): Sequential(
        (0): Linear(in_features=128, out_features=512, bias=True)
        (1): ReLU()
        (2): Linear(in_features=512, out_features=128, bias=True)
      )
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (dropout): Dropout(p=0.1, inplace=False)
)

In [19]:
from tqdm import tqdm

def generate_embeddings(model, loader):
    model.eval()
    embeddings = []
    with torch.no_grad():
        for batch, in tqdm(loader):
            batch = batch.to(device)
            embedding = model(batch)
            embeddings.append(embedding.cpu())
    return torch.cat(embeddings)

embeddings = generate_embeddings(model, loader)

100%|██████████| 3523/3523 [00:26<00:00, 131.65it/s]


In [20]:
embeddings = [emb.numpy() for emb in embeddings]
df['embeddings'] = embeddings

In [21]:
pd.set_option('display.max_colwidth', None)
df[['content','embeddings']].head(20)

,content,embeddings
0,Sleeimfh Vertical for a reason,"[-0.43363044, -0.7128726, 0.54620314, -1.3661556, -0.39889735, -1.6705528, -0.20286119, 0.26578686, -0.47472507, 2.0741549, -0.7649682, -0.1834096, 1.2377884, 0.22821067, -0.49831533, -0.5055845, 1.995737, -0.27213752, 0.060444057, -0.45454115, -1.3515954, -1.897715, 0.048867196, 0.71289873, -0.115548834, -1.0716354, 0.10862446, -0.8455559, 0.43479007, 0.53768516, -0.06648921, -0.99024534, 0.6547944, -1.3988183, -0.8555416, -0.7537482, -0.85873544, 0.6150476, 0.12027199, 0.25922558, -0.1799185, 0.4973003, 1.0898979, -1.8148155, -0.6585688, 1.0647717, -0.5442203, 0.56042904, -0.23637722, -1.6923996, -0.97342944, 1.5784436, 0.59828496, -1.3810647, 0.51643014, 0.9621568, 1.8787358, -0.031029398, 0.5312636, -0.29048365, 1.1259289, -0.288084, 0.011565836, 0.28617892, -1.4794569, -0.23959848, 0.5265198, 0.40754044, 1.1540188, -0.5650824, -0.4975955, 1.6474087, 0.40962392, -0.1527755, -0.34718382, 1.1029038, 0.5166485, 0.10627104, -0.39128387, 0.5464903, -1.17761, 0.23869619, -0.08403178, 0.36616838, 2.1437845, 0.76191926, -1.619201, -1.652935, -0.21456921, 1.0788082, 0.73933625, 0.9436886, -0.5013261, -0.46736625, -0.4921737, -0.84794503, -0.7269641, 2.6050308, -1.0265015, 0.86751044, ...]"
1,"I complained before, but now I'm pretty upset too. Netflix recently added a ""household"" function and now everything sucks. I can't get on Netflix while I'm traveling, at a friend's house, or even when I'm just using my data or a separate wifi from the TV the main account is connected to..Netflix needs to remove the household settings and let people watch where they wanna watch.","[-0.17959267, -0.8949485, -0.14898811, -0.5993668, -0.8894342, -0.8330884, -0.32284403, 0.2127953, -0.17926311, 0.6671449, -0.6142428, -0.75469226, 0.37036842, 0.06300043, -0.24585447, -0.3216456, 1.5351079, -0.3921864, 0.08347522, -0.43364578, -0.67961717, -0.7909801, 0.15953597, 0.37967, 0.0364741, -0.5816035, 0.32782108, -0.73634255, 0.06401537, 0.4280455, -0.36771294, -0.36311674, 0.47679764, -0.737763, -0.44084796, -0.29573423, -0.27391103, 0.7697592, -0.19138426, -0.5408249, -0.60155106, 0.32635626, -0.120862566, -1.4442458, -0.7405307, 0.38173452, -0.25933212, 0.37519902, 0.046910867, -1.2448336, -0.22473352, 0.7945496, 0.12216093, -1.1260087, 0.42541572, 0.61046505, 1.1346929, 0.20013739, 0.35319948, -0.12453724, 0.9830417, 0.4258144, 0.23614818, 0.55038714, -0.53836036, 0.3133121, 0.3678571, 0.59555715, -0.09875287, -0.00898154, -0.25439996, 1.5563487, -0.21222493, -0.21711083, 0.08630464, 0.6279125, 0.13120466, 0.05820298, -0.22833955, 0.6202475, -0.5762929, 0.22843096, -0.29366165, 0.5574071, 1.2177315, 0.53880745, -0.82504344, -0.60355794, 0.09574893, 0.32681704, 0.1029581, 1.1546193, -0.7404457, 0.12944378, -0.49679253, 0.56736416, -0.969064, 1.8798103, -1.2014625, 0.9152247, ...]"
2,On top of everything that has been going down in the last years... Where the heck is the CAST button now?,"[-0.3208459, -0.74053746, 0.43258095, -1.238201, -0.49313188, -1.6121296, -0.28042433, 0.33578843, -0.47388846, 1.8385084, -0.7381846, -0.24936715, 1.1358099, 0.247514, -0.5248021, -0.49863183, 1.9083625, -0.28454822, 0.0776687, -0.49844792, -1.2356344, -1.7071269, 0.09634858, 0.66002405, -0.059748694, -1.0454088, 0.13176946, -0.82210964, 0.43314028, 0.49649173, -0.07221107, -0.85642123, 0.6293974, -1.2977237, -0.8882248, -0.6823874, -0.79625463, 0.7115346, 0.06393953, 0.12494548, -0.289447, 0.41302, 0.89283806, -1.7412238, -0.62652284, 1.0133507, -0.4667846, 0.55691314, -0.21736729, -1.7416039, -0.82560796, 1.4284604, 0.505738, -1.3303928, 0.47540954, 0.8901236, 1.7581313, -0.050470002, 0.5261601, -0.25257978, 1.1669621, -0.22133397, 0.029182015, 0.29774982, -1.328937, -0.14082131, 0.52916765, 0.41448, 0.9302147, -0.4626455, -0.5553121, 1.7052883, 0.30353287, -0.2230578, -0.28602123, 1.1145897, 0.5267215, 0.09493868, -0.33365586, 0.5505352, -1.0861197, 0.2584845, -0.11239966, 0.40080136, 2.0258155, 0.73761183, -1.5